In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup

# URL of the Home Depot category page you want to scrape
HOME_DEPOT_URL = 'https://www.homedepot.com/b/Tools/N-5yc1vZc1xy'

def get_homedepot_products(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an error for bad responses
    soup = BeautifulSoup(response.text, 'html.parser')

    products = []
    # Adjust the selectors based on the actual HTML structure of the Home Depot page
    for item in soup.select('.product-details'):
        product_name = item.select_one('.product-title').get_text(strip=True)
        product_price = item.select_one('.price').get_text(strip=True).replace('$', '').replace(',', '')
        
        products.append({
            'name': product_name,
            'price': float(product_price) if product_price else 0.0
        })

    return products

def main():
    products = get_homedepot_products(HOME_DEPOT_URL)
    for product in products:
        print(f"Product Name: {product['name']}, Price: ${product['price']:.2f}")

if __name__ == "__main__":
    main()

In [3]:
import requests
from bs4 import BeautifulSoup
import time

# Constants
HOME_DEPOT_URL = 'https://www.homedepot.com/b/Tools/N-5yc1vZc1xy'  # Example category
WALMART_API_URL = 'https://marketplace.walmart.com/v3/listing'
WALMART_API_KEY = 'your_walmart_api_key'
TAX_RATE = 0.07  # Example tax rate of 7%
WALMART_FEES = 0.15  # Example Walmart fees of 15%
DESIRED_ROI = 1.30  # 30% ROI

def get_homedepot_products(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'
    }
    products = []
    
    while url:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        for item in soup.select('.product-details'):
            product_name = item.select_one('.product-title').get_text(strip=True)
            product_price = item.select_one('.price').get_text(strip=True).replace('$', '').replace(',', '')
            product_sku = item.get('data-sku')  # Adjust based on actual structure
            
            if product_price and product_sku:
                products.append({
                    'name': product_name,
                    'price': float(product_price),
                    'sku': product_sku
                })

        # Find next page link (if exists)
        next_page = soup.select_one('.pagination-next a')
        url = next_page['href'] if next_page else None
        time.sleep(1)  # Sleep to avoid overwhelming the server

    return products

def calculate_selling_price(cost_price):
    total_cost = cost_price * (1 + TAX_RATE)  
    selling_price = total_cost * (1 + WALMART_FEES)
    return selling_price * DESIRED_ROI

def list_product_on_walmart(product):
    headers = {
        'Authorization': f'Bearer {WALMART_API_KEY}',
        'Content-Type': 'application/json'
    }
    data = {
        'sku': product['sku'],
        'name': product['name'],
        'price': product['selling_price'],
        'description': f"{product['name']} - Check out this product!",
        # Add more fields as required by Walmart's API
    }
    response = requests.post(WALMART_API_URL, headers=headers, json=data)
    response.raise_for_status()
    return response.json()

def main():
    products = get_homedepot_products(HOME_DEPOT_URL)
    for product in products:
        cost_price = product['price']
        selling_price = calculate_selling_price(cost_price)

        if selling_price > cost_price:
            product_data = {
                'sku': product['sku'],
                'name': product['name'],
                'selling_price': selling_price,
            }
            try:
                response = list_product_on_walmart(product_data)
                print(f"Listed {product['name']} on Walmart: {response}")
            except Exception as e:
                print(f"Failed to list {product['name']} on Walmart: {e}")

if __name__ == "__main__":
    main()